# 第4章 朴素贝叶斯

1．朴素贝叶斯法是典型的生成学习方法。生成方法由训练数据学习联合概率分布
$P(X,Y)$，然后求得后验概率分布$P(Y|X)$。具体来说，利用训练数据学习$P(X|Y)$和$P(Y)$的估计，得到联合概率分布：

$$P(X,Y)＝P(Y)P(X|Y)$$

概率估计方法可以是极大似然估计或贝叶斯估计。

2．朴素贝叶斯法的基本假设是条件独立性，

$$\begin{aligned} P(X&=x | Y=c_{k} )=P\left(X^{(1)}=x^{(1)}, \cdots, X^{(n)}=x^{(n)} | Y=c_{k}\right) \\ &=\prod_{j=1}^{n} P\left(X^{(j)}=x^{(j)} | Y=c_{k}\right) \end{aligned}$$


这是一个较强的假设。由于这一假设，模型包含的条件概率的数量大为减少，朴素贝叶斯法的学习与预测大为简化。因而朴素贝叶斯法高效，且易于实现。其缺点是分类的性能不一定很高。

3．朴素贝叶斯法利用贝叶斯定理与学到的联合概率模型进行分类预测。

$$P(Y | X)=\frac{P(X, Y)}{P(X)}=\frac{P(Y) P(X | Y)}{\sum_{Y} P(Y) P(X | Y)}$$
 
将输入$x$分到后验概率最大的类$y$。

$$y=\arg \max _{c_{k}} P\left(Y=c_{k}\right) \prod_{j=1}^{n} P\left(X_{j}=x^{(j)} | Y=c_{k}\right)$$

后验概率最大等价于0-1损失函数时的期望风险最小化。


模型：

- 高斯模型
- 多项式模型
- 伯努利模型

## 1. 实现朴素贝叶斯

In [39]:
import numpy as np

X = np.array([
    [1,1,0],
    [0,0,1],
    [1,0,1],
    [1,0,0],
    [0,1,0],
    [0,1,1],
    [0,1,0],
    [0,0,1]
])

y = np.array([1,0,0,1,1,0,0,1])


In [40]:
# 朴素贝叶斯类，面向离散特征
class NaiveBayes:
    def __init__(self):
        self.prior = {}
        self.prior_num = {}
        self.posterior = {}    

    # 训练模型
    def fit(self, X, y):
        
        # 计算先验概率  数据结构形式：{0: 0.5, 1: 0.5}
        labels = list(set(y))
        self.prior = {label: 0 for label in labels}  
        self.prior_num = {label: 0 for label in labels}  
        for value in y:            
            self.prior[value] +=1.0/ len(y) #累计计算先验概率
            self.prior_num[value] += 1 #累计不同分类的样本个数，备用
        
        # 计算后验概率  数据结构形式：{0: [{0: 0.75, 1: 0.25}, {0: 0.5, 1: 0.5}......}
        self.posterior = {label: [] for label in labels}
        for label in labels:
            for _ in range(X.shape[-1]):
                self.posterior[label].append({}) # 为每个类别，初始化一个空字典 
        for item, label in zip(X, y):
            prior_num_y = self.prior_num[label]
            for i, val in enumerate(item):
                if val in self.posterior[label][i]:  
                    self.posterior[label][i][val] += 1.0/prior_num_y  #已存在该特征值，则进行累计计算
                else:
                    self.posterior[label][i][val] = 1.0/prior_num_y  #不存在该特征值，则初始化        
        print("先验概率:",self.prior)
        print("后验概率:",self.posterior)
    
    # 通过已知的概率，对样本做预测
    def predict_single(self, X_test):
        results = {}
        # 通过朴素贝叶斯公式来计算
        for label,prior_val in self.prior.items():
            results[label] = prior_val
            for i, post_val in enumerate(X_test): 
                results[label] *= self.posterior[label][i][post_val]
        denominator = np.sum(list(results.values()))
        # 返回不同分类的概率
        for label in results.keys(): 
            results[label] = results[label]/denominator
        return results


In [41]:
nb = NaiveBayes()
nb.fit(X,y)
test_sample = [0,0,1]
print("测试结果:", nb.predict_single(test_sample))

先验概率: {0: 0.5, 1: 0.5}
后验概率: {0: [{0: 0.75, 1: 0.25}, {0: 0.5, 1: 0.5}, {1: 0.75, 0: 0.25}], 1: [{1: 0.5, 0: 0.5}, {1: 0.5, 0: 0.5}, {0: 0.75, 1: 0.25}]}
测试结果: {0: 0.8181818181818182, 1: 0.18181818181818182}


## 2、使用Sklearn计算

In [42]:
# 需要使用sklearn版本 0.22以上
from sklearn.naive_bayes import CategoricalNB

mnb = CategoricalNB(alpha=0.00001)
mnb.fit(X,y)
test_sample = np.array([[0,0,1]])
print("测试结果:", mnb.predict_proba(test_sample))

测试结果: [[0.81818058 0.18181942]]



## 3. GaussianNB 高斯朴素贝叶斯

特征的可能性被假设为高斯

概率密度函数：
$$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$

方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

In [43]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [44]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[50:150, :])
    # print(data)
    return data[:,:-1], data[:,-1]

In [45]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_test[0], y_test[0]

(array([6.8, 3.2, 5.9, 2.3]), 2.0)

In [53]:
class NaiveBayes:
    def __init__(self):
        self.model = None

    # 数学期望
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))

    # 标准差（方差）
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X)))

    # 概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent

    # 处理X_train
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in np.array(train_data).T]
        return summaries

    # 分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))
        data = {label: [] for label in labels}  # {0: [], 1: []}
        for f, label in zip(X, y):
            data[label].append(f)
        self.model = {   
            label: self.summarize(value) 
            for label, value in data.items()
        } # {0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        return 'gaussianNB train done!'

    # 计算概率
    def calculate_probabilities(self, input_data):
        # summaries:{0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        # input_data:[1.1, 2.2]
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(
                    input_data[i], mean, stdev)
        return probabilities

    # 类别
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]
        return label

    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [54]:
model = NaiveBayes()
model.fit(X_train, y_train)
print(model.predict([4.4,  3.2,  1.3,  0.2]))
model.score(X_test, y_test)

1.0


0.8666666666666667

## 4、高斯朴素贝叶斯 scikit-learn实例

In [48]:
from sklearn.naive_bayes import GaussianNB

In [49]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [50]:
clf.score(X_test, y_test)

0.8666666666666667

In [25]:
clf.predict([[4.4,  3.2,  1.3,  0.2]])

array([1.])